In [25]:
!pwd

/c/Users/sarawute/Desktop/6DRepNet


In [31]:
# goto path 6DRepNet
%cd C:\Users\sarawute\Desktop\6DRepNet

C:\Users\sarawute\Desktop\6DRepNet


In [18]:
!venv\Scripts\activate

In [19]:
!pip install -r requirements1.txt

In [20]:
!pip install git+https://github.com/elliottzheng/face-detection.git@master

^C


In [21]:
!pip install scipy

In [22]:
!pip install opencv-python

  Running command git clone --filter=blob:none --quiet https://github.com/elliottzheng/face-detection.git 'C:\Users\sarawute\AppData\Local\Temp\pip-req-build-equnluo3'


  Cloning https://github.com/elliottzheng/face-detection.git (to revision master) to c:\users\sarawute\appdata\local\temp\pip-req-build-equnluo3
  Resolved https://github.com/elliottzheng/face-detection.git to commit 096cbf438dab07a313dac3044653d211725d64ba
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [23]:
!pip install matplotlib

In [9]:
!python demo.py  --snapshot 6DRepNet_300W_LP_AFLW2000.pth --cam 0

In [ ]:
from face_detection import RetinaFace
from model import SixDRepNet
import math
import re
from matplotlib import pyplot as plt
import sys
import os
import argparse
import pprint
import re

import numpy as np
import cv2
import matplotlib.pyplot as plt
from numpy.lib.function_base import _quantile_unchecked

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
import utils
import matplotlib
from PIL import Image

matplotlib.use('TkAgg')
from collections import OrderedDict




transformations = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

if __name__ == '__main__':

    cudnn.enabled = True
    cam = 0
    snapshot_path = "C:\\Users\\sarawute\\Desktop\\6DRepNet\\6DRepNet_300W_LP_AFLW2000.pth"
    model = SixDRepNet(backbone_name='RepVGG-B1g2',
                       backbone_file='',
                       deploy=True,
                       pretrained=False)

    print('Loading data.')


    detector = RetinaFace()





    # Load snapshot
    saved_state_dict = torch.load(os.path.join(
        snapshot_path), map_location='cpu')
    
    print(saved_state_dict.keys())
    


    if 'model_state_dict' in saved_state_dict:

        model.load_state_dict(saved_state_dict['model_state_dict'])
    else:
        model.load_state_dict(saved_state_dict)

    # Test the Model
    model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).

    cap = cv2.VideoCapture(cam)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    with torch.no_grad():
        while True:
            ret, frame = cap.read()

            faces = detector(frame)

            for box, landmarks, score in faces:

                # Print the location of each face in this image
                if score < .95:
                    continue
                x_min = int(box[0])
                y_min = int(box[1])
                x_max = int(box[2])
                y_max = int(box[3])
                bbox_width = abs(x_max - x_min)
                bbox_height = abs(y_max - y_min)

                x_min = max(0, x_min-int(0.2*bbox_height))
                y_min = max(0, y_min-int(0.2*bbox_width))
                x_max = x_max+int(0.2*bbox_height)
                y_max = y_max+int(0.2*bbox_width)

                img = frame[y_min:y_max, x_min:x_max]
                img = Image.fromarray(img)
                img = img.convert('RGB')
                img = transformations(img)

                img = torch.Tensor(img[None, :])

                c = cv2.waitKey(1)
                if c == 27:
                    break

                R_pred = model(img)
   

                euler = utils.compute_euler_angles_from_rotation_matrices(
                    R_pred)*180/np.pi
                p_pred_deg = euler[:, 0].cpu()
                y_pred_deg = euler[:, 1].cpu()
                r_pred_deg = euler[:, 2].cpu()

                #utils.draw_axis(frame, y_pred_deg, p_pred_deg, r_pred_deg, left+int(.5*(right-left)), top, size=100)
                utils.plot_pose_cube(frame,  y_pred_deg, p_pred_deg, r_pred_deg, x_min + int(.5*(
                    x_max-x_min)), y_min + int(.5*(y_max-y_min)), size=bbox_width)

            cv2.imshow("Demo", frame)
            cv2.waitKey(5)


Loading data.
odict_keys(['layer0.rbr_reparam.weight', 'layer0.rbr_reparam.bias', 'layer1.0.rbr_reparam.weight', 'layer1.0.rbr_reparam.bias', 'layer1.1.rbr_reparam.weight', 'layer1.1.rbr_reparam.bias', 'layer1.2.rbr_reparam.weight', 'layer1.2.rbr_reparam.bias', 'layer1.3.rbr_reparam.weight', 'layer1.3.rbr_reparam.bias', 'layer2.0.rbr_reparam.weight', 'layer2.0.rbr_reparam.bias', 'layer2.1.rbr_reparam.weight', 'layer2.1.rbr_reparam.bias', 'layer2.2.rbr_reparam.weight', 'layer2.2.rbr_reparam.bias', 'layer2.3.rbr_reparam.weight', 'layer2.3.rbr_reparam.bias', 'layer2.4.rbr_reparam.weight', 'layer2.4.rbr_reparam.bias', 'layer2.5.rbr_reparam.weight', 'layer2.5.rbr_reparam.bias', 'layer3.0.rbr_reparam.weight', 'layer3.0.rbr_reparam.bias', 'layer3.1.rbr_reparam.weight', 'layer3.1.rbr_reparam.bias', 'layer3.2.rbr_reparam.weight', 'layer3.2.rbr_reparam.bias', 'layer3.3.rbr_reparam.weight', 'layer3.3.rbr_reparam.bias', 'layer3.4.rbr_reparam.weight', 'layer3.4.rbr_reparam.bias', 'layer3.5.rbr_repa